In [8]:
# Подключение к Mongo DB и запись коллекции: 
from pymongo import MongoClient

client = MongoClient(
    "mongodb://ich_editor:verystrongpassword"
    "@mongo.itcareerhub.de/?readPreference=primary"
    "&ssl=false&authMechanism=DEFAULT&authSource=ich_edit"
)

# Выбор базы данных Mongo DB
db = client["ich_edit"]

# Выбор коллекции 
collection = db["final_project_dam130625_H_Ovsiannikova"]

from datetime import datetime
def current_datetime():
    return datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
def write_to_mongo(results, query_type, params):
    """Сохраняет результаты поиска в MongoDB"""
    doc = {
        "timestamp": current_datetime(),
        "query_type": query_type,
        "params": params,
        "count": len(results),
        "results": results
    }
    collection.insert_one(doc)
    print("Результаты поиска сохранены в MongoDB.")

In [9]:
# Модуль статистика

def show_last_queries(limit=5):
    """Показать последние N запросов"""
    last_queries = list(collection.find().sort("_id", -1).limit(limit))
    if not last_queries:
        print("Нет сохранённых запросов.")
        return

    print(f"\nПоследние {len(last_queries)} запросов:")
    for i, entry in enumerate(last_queries, 1):
        qtype = entry.get("query_type")
        params = entry.get("params", {})

        if qtype == "keyword":
            display = f"По ключевому слову '{params.get('keyword')}'"
        elif qtype == "genre_year":
            display = f"По жанру {params.get('category_id')} и годам {params.get('year_from')}-{params.get('year_to')}"
        else:
            display = str(params)

        print(f"{i}. {display}")

def show_popular_queries():
    pipeline = [
        {"$group": {"_id": {"type": "$query_type", "params": "$params"}, "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {"$limit": 5}
    ]
    popular = list(collection.aggregate(pipeline))

    if not popular:
        print("Нет данных о популярных запросах.")
        return

    print("\nТОП-5 популярных запросов:")
    for i, item in enumerate(popular, 1):
        qtype = item["_id"]["type"]
        params = item["_id"]["params"]

        if qtype == "keyword":
            display = f"По ключевому слову '{params.get('keyword')}'"
        elif qtype == "genre_year":
            display = f"По жанру {params.get('category_id')} и годам {params.get('year_from')}-{params.get('year_to')}"
        else:
            display = str(params)

        print(f"{i}. {display} — {item['count']} раз")


In [11]:
# Создание соединения к SQL database = 'sakila'

import mysql.connector

conn = mysql.connector.connect(
    host = 'ich-db.edu.itcareerhub.de',
    port = '3306',
    user = 'ich1',
    password = 'password',
    database = 'sakila'
    )

## Проверка соединения
if conn.is_connected():
    print("Есть соединение с базой sakila.")
else:
    print ("Соединение не установлено")

cursor = conn.cursor() # Cоздание курсора для работы с SQL запросами

Есть соединение с базой sakila.


In [12]:
# Основные внутренние функции поиска в базе SQL

def show_categories(cursor):
    cursor.execute("""
        SELECT category_id,
               name
        FROM category
        ORDER BY name;
    """)
    rows = cursor.fetchall()

    print("\n=== Список жанров ====")
    for category_id, name in rows:
        print(f"{category_id}: {name}")
    print("======================\n")

    return rows

def show_year_range(cursor):
    cursor.execute("""
        SELECT MIN(release_year), MAX(release_year)
        FROM film;
    """)
    
    min_year, max_year = cursor.fetchone()
    
    print(f"\nГод выпуска фильмов: от {min_year} до {max_year}\n")
    
    return min_year, max_year

def search_by_keyword(cursor):
    keyword = input("Введите ключевое слово для поиска: ")
    cursor.execute(
        """
        SELECT title,
               release_year
        FROM film
        WHERE title LIKE %s
        ORDER BY title;
        """,
        (f"%{keyword}%",)
    )
    results = cursor.fetchall()
    return results, keyword  # возвращаем два значения

def search_by_genre_and_year(cursor):
    category_id = input("Введите жанр (цифру): ")
    year_from = input("Введите год от: ")
    year_to = input("Введите год до: ")

    query = """
        SELECT f.title,
               f.release_year,
               c.name
        FROM film AS f
        LEFT JOIN film_category AS fc ON f.film_id = fc.film_id
        LEFT JOIN category AS c ON fc.category_id = c.category_id
        WHERE c.category_id = %s
          AND f.release_year BETWEEN %s AND %s
        ORDER BY f.release_year;
    """

    cursor.execute(query, (category_id, year_from, year_to))
    results = cursor.fetchall()
    return results, category_id, year_from, year_to  # возвращаем параметры вместе с результатами

In [15]:
# Функция пагинации (пролистывания страниц)

def show_paginated(results):
    index = 0
    total =len(results)
    if total == 0:
        print ("Ничего не найдено.")
        return

    while index < total:
        print("\n=== Результаты ===")
        for i in range(index, min(index + 10, total)):
            print(results[i])
        print("==============")

        index += 10
        if index < total:
            cont = input("Показать следующие 10? (да/нет): ")
            if cont.lower() != 'да':
                break
                

In [ ]:
# Основной блок (интерактивное меню)

def main():
    show_categories(cursor)
    show_year_range(cursor)

    while True:
        print("\nВыберите тип поиска:")
        print("1 - Поиск по ключевому слову")
        print("2 - Поиск по жанру и диапазону годов")
        print("3 - Показать последние 5 запросов")
        print("4 - Показать ТОП-5 популярных запросов")
        print("0 - Выход")

        choice = input("Ваш выбор: ")

        if choice == "1":
            results, keyword = search_by_keyword(cursor)
            show_paginated(results)
            write_to_mongo(results, "keyword", {"keyword": keyword})

        elif choice == "2":
            results, category_id, year_from, year_to = search_by_genre_and_year(cursor)
            show_paginated(results)
            write_to_mongo(
                results,
                "genre_year",
                {"category_id": category_id, "year_from": year_from, "year_to": year_to}
            )

        elif choice == "3":
            show_last_queries()

        elif choice == "4":
            show_popular_queries()

        elif choice == "0":
            break

        else:
            print("Неверный ввод, попробуйте еще раз.")

    cursor.close()
    conn.close()
    client.close()
    print("Выход из программы.")


if __name__ == "__main__":
    main()



=== Список жанров ====
1: Action
2: Animation
3: Children
4: Classics
5: Comedy
6: Documentary
7: Drama
8: Family
9: Foreign
10: Games
11: Horror
12: Music
13: New
14: Sci-Fi
15: Sports
16: Travel


Год выпуска фильмов: от 1990 до 2025


Выберите тип поиска:
1 - Поиск по ключевому слову
2 - Поиск по жанру и диапазону годов
3 - Показать последние 5 запросов
4 - Показать ТОП-5 популярных запросов
0 - Выход


Ваш выбор:  3



Последние 5 запросов:
1. По ключевому слову 'Ang'
2. По ключевому слову 'ab'
3. По жанру 5 и годам 1990-2000
4. По ключевому слову 'egg'
5. По жанру 7 и годам 2004-2020

Выберите тип поиска:
1 - Поиск по ключевому слову
2 - Поиск по жанру и диапазону годов
3 - Показать последние 5 запросов
4 - Показать ТОП-5 популярных запросов
0 - Выход


Ваш выбор:  4



ТОП-5 популярных запросов:
1. По ключевому слову '111' — 7 раз
2. По жанру 7 и годам 1996-1997 — 1 раз
3. По ключевому слову 'Anna' — 1 раз
4. По ключевому слову 'Egg' — 1 раз
5. По жанру 11 и годам 1990-1995 — 1 раз

Выберите тип поиска:
1 - Поиск по ключевому слову
2 - Поиск по жанру и диапазону годов
3 - Показать последние 5 запросов
4 - Показать ТОП-5 популярных запросов
0 - Выход


...


Последние 5 запросов:
1. По жанру 14 и годам 1995-2000
2. По ключевому слову 'Ang'
3. По ключевому слову 'ab'
4. По жанру 5 и годам 1990-2000
5. По ключевому слову 'egg'

Выберите тип поиска:
1 - Поиск по ключевому слову
2 - Поиск по жанру и диапазону годов
3 - Показать последние 5 запросов
4 - Показать ТОП-5 популярных запросов
0 - Выход


Ваш выбор:  1
Введите ключевое слово для поиска:  Egg



=== Результаты ===
('AFRICAN EGG', 1998)
('EGG IGBY', 2009)
('RACER EGG', 2024)
Результаты поиска сохранены в MongoDB.

Выберите тип поиска:
1 - Поиск по ключевому слову
2 - Поиск по жанру и диапазону годов
3 - Показать последние 5 запросов
4 - Показать ТОП-5 популярных запросов
0 - Выход


Ваш выбор:  3



Последние 5 запросов:
1. По ключевому слову 'Egg'
2. По жанру 14 и годам 1995-2000
3. По ключевому слову 'Ang'
4. По ключевому слову 'ab'
5. По жанру 5 и годам 1990-2000

Выберите тип поиска:
1 - Поиск по ключевому слову
2 - Поиск по жанру и диапазону годов
3 - Показать последние 5 запросов
4 - Показать ТОП-5 популярных запросов
0 - Выход
